In [22]:
import matplotlib.pyplot as plt  # type: ignore
# 로컬에 저장된 모델 불러오기
save_path = "/home/oss_1/MinsuKim/stable-diffusion-3.5-model"
pipe = StableDiffusion3Pipeline.from_pretrained(save_path, torch_dtype=torch.bfloat16)

pipe = pipe.to("cuda")

Loading pipeline components...:   0%|          | 0/9 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

['__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_best_guess_weight_name', '_callback_tensor_inputs', '_deprecated_kwargs', '_dict_from_json_file', '_exclude_from_cpu_offload', '_execution_device', '_fetch_state_dict', '_get_clip_prompt_embeds', '_get_init_keys', '_get_signature_keys', '_get_signature_types', '_get_t5_prompt_embeds', '_internal_dict', '_is_onnx', '_load_connected_pipes', '_lora_loadable_modules', '_optional_components', '_optionally_disable_offloading', '_upload_folder', 'check_inputs', 'clip_skip', 'components', 'config', 'config_name', 'default_sample_size', 'delete_adapters', 'device', 'disable_attention_slicing', 'disable_lora', 'disable_xformers_

: 

: 

In [83]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

# 데이터셋 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, image_root, label_root, transform=None):
        self.image_root = image_root
        self.label_root = label_root
        self.transform = transform
        self.image_paths = []
        self.label_paths = []

        # 이미지와 레이블 파일 경로를 일치시키는 작업
        for image_folder in os.listdir(image_root):
            if image_folder.endswith('.zip') or image_folder == '.ipynb_checkpoints':
                continue
            image_folder_path = os.path.join(image_root, image_folder)
            label_folder_path = os.path.join(label_root, f"TL_TEXT_{image_folder.split('_')[1]}_{image_folder.split('_')[2]}")

            for image_file in os.listdir(image_folder_path):
                label_file = image_file.split(".")[0] + ".txt"
                self.image_paths.append(os.path.join(image_folder_path, image_file))
                self.label_paths.append(os.path.join(label_folder_path, label_file))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label_path = self.label_paths[idx]

        # 이미지 로드 및 전처리
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        # 레이블 로드
        with open(label_path, "r", encoding="utf-8") as file:
            label = file.read().strip()  # 레이블 파일 내용 읽기

        return image, label

# 이미지 전처리 설정 (1024x1024 크기로 조정, 정규화)
transform = transforms.Compose([
    transforms.Resize((1024, 1024)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# 데이터셋 및 DataLoader 생성
train_image_root = "/home/oss_1/data/240.심볼(로고) 생성 데이터/01-1.정식개방데이터/Training/01.원천데이터"
train_label_root = "/home/oss_1/data/240.심볼(로고) 생성 데이터/01-1.정식개방데이터/Training/02.라벨링데이터"

train_dataset = CustomDataset(train_image_root, train_label_root, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [14]:
   print("Number of Image Paths:", len(train_dataset.image_paths))
   print("Number of Label Paths:", len(train_dataset.label_paths))
   print("Sample Image Paths:", train_dataset.image_paths[:10])  # 처음 10개 경로만 출력
   print("Sample Label Paths:", train_dataset.label_paths[:10])  # 처음 10개 경로만 출력

Number of Image Paths: 466984
Number of Label Paths: 466984
Sample Image Paths: ['/home/oss_1/data/240.심볼(로고) 생성 데이터/01-1.정식개방데이터/Training/01.원천데이터/TS_03.combinationmark_P/4120040027696.png', '/home/oss_1/data/240.심볼(로고) 생성 데이터/01-1.정식개방데이터/Training/01.원천데이터/TS_03.combinationmark_P/4020200016592.png', '/home/oss_1/data/240.심볼(로고) 생성 데이터/01-1.정식개방데이터/Training/01.원천데이터/TS_03.combinationmark_P/4120010000416.png', '/home/oss_1/data/240.심볼(로고) 생성 데이터/01-1.정식개방데이터/Training/01.원천데이터/TS_03.combinationmark_P/4020190018779.png', '/home/oss_1/data/240.심볼(로고) 생성 데이터/01-1.정식개방데이터/Training/01.원천데이터/TS_03.combinationmark_P/4019880029302.png', '/home/oss_1/data/240.심볼(로고) 생성 데이터/01-1.정식개방데이터/Training/01.원천데이터/TS_03.combinationmark_P/4119960008542.png', '/home/oss_1/data/240.심볼(로고) 생성 데이터/01-1.정식개방데이터/Training/01.원천데이터/TS_03.combinationmark_P/4120150012485.png', '/home/oss_1/data/240.심볼(로고) 생성 데이터/01-1.정식개방데이터/Training/01.원천데이터/TS_03.combinationmark_P/4120150022924.png', '/home/oss_1/data/240.심볼(로고) 생성

In [79]:
print(train_dataset.image_paths)
print(train_dataset.label_paths)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

